In [15]:
from torch import tensor
from torchmetrics.classification import MulticlassAccuracy
from src.trainer.metrics import Metrics

In [19]:
target = tensor([2, 2, 1, 0, 0])
preds = tensor([2, 1, 1, 0, 1])
metric = Metrics(3, 1).mca
metric.update(preds, target)
metric.compute()

tensor(0.6667)

In [12]:
metric.fn, metric.tp, metric.fp, metric.tn

(tensor([1, 0, 0]), tensor([1, 1, 1]), tensor([0, 1, 0]), tensor([2, 2, 3]))

In [13]:
(metric.tp.sum()) / (metric.tp.sum() + metric.fn.sum())

tensor(0.7500)